In [59]:
import altair as alt, pandas as pd, requests, json

import sys
sys.path.append('../')
from altair_wrapper.eco_styles import EcoStyles

styles = EcoStyles()

In [2]:
def get_ons_data(series_id):
    dataset_id='MGDP'

    # Use ONS API to get monthly data
    url = f'https://api.allorigins.win/raw?url=https://api.ons.gov.uk/timeseries/{series_id}/dataset/{dataset_id}/data'
    # read data at api into dataframe, relevant data is in the 'months' key
    # Make a GET request to fetch the raw JSON content
    json_data = requests.get(url).json()

    # Extract data from the 'months' key
    months_data = json_data['months']

    # Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame.from_dict(months_data)

    # Clean data set, convert date from yyyy mmm to yyyy-mm-dd
    df['date'] = pd.to_datetime(df['date'], format='%Y %b')

    # drop unnecessary columns
    df.drop(columns=['label', 'quarter', 'sourceDataset', 'updateDate'], inplace=True)

    # convert value column to float
    df['value'] = df['value'].astype(float)

    return df

In [3]:
df_ecy2 = get_ons_data(series_id='ecy2')

In [4]:
df_ecy2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    322 non-null    datetime64[ns]
 1   month   322 non-null    object        
 2   value   322 non-null    float64       
 3   year    322 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 10.2+ KB


In [37]:
dark = True

background = '#122b39' if dark else '#b4c8d8'
detail = '#b4c8d8' if dark else '#122b39'
line_colour = '#36b7b4'

In [42]:
df_ecy2[df_ecy2['date'] >= '2019-01-01']

,date,month,value,year
264,2019-01-01,January,98.9,2019
265,2019-02-01,February,99.5,2019
266,2019-03-01,March,99.6,2019
267,2019-04-01,April,99.3,2019
268,2019-05-01,May,99.6,2019
269,2019-06-01,June,100.1,2019
270,2019-07-01,July,100.6,2019
271,2019-08-01,August,100.3,2019
272,2019-09-01,September,100.6,2019
273,2019-10-01,October,100.5,2019


In [89]:
(102.4 -102.1) / 102.1

0.0029382957884428146

In [40]:
# create chart using Altair

start_date = '2016-01-01'

chart = alt.Chart(df_ecy2[df_ecy2['date'] >= start_date]).mark_line(color=line_colour).encode(
    x=alt.X(
        'date:T',
        axis=alt.Axis(
            labelColor=detail,
            tickColor=detail,
            domainColor=detail,
            domainOpacity=0.5,
            grid=False,
            labelAngle=0,
            tickCount=10,
            tickOpacity=0.5,
            title=None
        )
    ),
    y=alt.Y(
        'value:Q',
        axis=alt.Axis(
            labelColor=detail,
            tickColor=detail,
            domainColor=detail,
            gridColor=detail,
            gridDash=[1, 5],
            gridOpacity=0.5,
            labelPadding=5,
            tickCount=8,
            tickOpacity=0.5,
            ticks=False,
            title='Monthly index, 2019 = 100 | Source: ONS',
            titleAlign='left',
            titleAngle=0,
            titleBaseline='bottom',
            titleColor=detail,
            titleFontSize=12,
            titleOpacity=0.9,
            titleX=0,
            titleY=-7
        ),
        scale=alt.Scale(zero=False, padding=45)
    )
).properties(
    width = 400,
    height = 300,
    title = 'UK GDP'
)

# Add a point on at the last data point
# First filter the data to get the last data point
# Then add a point on the chart
last_point = df_ecy2[df_ecy2['date'] == df_ecy2['date'].max()].copy()

point = alt.Chart(last_point).mark_point(color=line_colour, size=60, opacity=0.9, filled=True).encode(
    x=alt.X('date:T'),
    y=alt.Y('value:Q')
)

# Now let's add text to the point
# add a column to `last_point` dataframe with the text we want to display
last_point['text'] = last_point['value'].apply(lambda x: f'{x:.1f}')

text_value = point.mark_text(
    align='left',
    baseline='middle',
    dx=7,
    dy=0,
    color=detail,
    fontSize=14,
).encode(
    text='text'
)

text_month = point.mark_text(
    align='left',
    baseline='middle',
    dx=7,
    dy=-16,
    color=detail,
    fontSize=14,
).encode(
    text='month'
)

chart = chart + point + text_month + text_value


chart = chart.configure(
    font='Circular Std',
    background=background
).configure_title(
    anchor='start',
    dx=20,
    fontSize=12,
    subtitleFontSize=12,
    color=detail,
    subtitleColor='#000000'
).configure_view(
    stroke=None
)

chart.display()

alt.LayerChart(...)

---

Gross Value Added - Monthly (period on period growth): CVM SA

In [30]:
# Monthly (period on period growth)
df_ecyx = get_ons_data(series_id='ecyx')

# Gross Value Added - Monthly (3 month on 3 month growth) :CVM SA
df_ed3h = get_ons_data(series_id='ed3h')

In [69]:
bar_positive = '#36b7b4'
bar_neagtive = '#e6224b'

subtitle_colour = detail + 'E6'

In [87]:
# Create bar chart of recent monthly GDP growth

start_date = '2022-01-01'

title = alt.Title(
    text='UK GDP Growth',
    subtitle='Monthly GDP Growth, % | Source: ONS',
    color=detail,
    subtitleColor=subtitle_colour,
    dx=27,
    fontSize=14,
    subtitleFontSize=12,
    offset=0
)

chart = alt.Chart(df_ecyx[df_ecyx['date'] >= start_date], title=title).mark_bar(width=13).encode(
    x=alt.X(
        'date:T',
        axis=alt.Axis(
            labelColor=detail,
            labelFontSize=11,
            tickColor=detail,
            domainColor=detail,
            domainOpacity=0.5,
            grid=False,
            labelAngle=0,
            tickCount=10,
            tickOpacity=0.5,
            title=None
        ),
        scale=alt.Scale(padding=20)
    ),
    y=alt.Y(
        'value:Q',
        axis=alt.Axis(
            labelColor=detail,
            labelFontSize=11,
            tickColor=detail,
            domainColor=detail,
            gridColor=detail,
            gridDash=[1, 5],
            gridOpacity=0.5,
            labelPadding=5,
            tickCount=7,
            tickOpacity=0.5,
            ticks=False,
            title=None

        ),
        scale=alt.Scale(zero=False, padding=35)
    ),
    # conditionally set colour of bar positive or negative value
    color=alt.condition(
        alt.datum.value > 0,
        alt.value(bar_positive),
        alt.value(bar_neagtive)
    )
).properties(
    width = 380,
    height = 300
)


########

# add a line at y=0
chart += alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color=detail, size=1.5, opacity=0.7).encode(y='y')

chart = chart.configure(
    font='Circular Std',
    background=background
).configure_title(
    anchor='start'
).configure_view(
    stroke=None
)


######

# Add a point on at the last data point
# First filter the data to get the last data point
# Then add a point on the chart
last_point = df_ecyx[df_ecyx['date'] == df_ecyx['date'].max()].copy()

# text_value = alt.Chart(last_point).mark_point(color=bar_positive, size=60, opacity=0.9, filled=True).encode(
#     x=alt.X('date:T'),
#     y=alt.Y('value:Q')
# )

# Now let's add text to the point
# add a column to `last_point` dataframe with the text we want to display
last_point['text'] = last_point['value'].apply(lambda x: f'{x:.1f}%')

text_value = alt.Chart(last_point).mark_text(
    align='left',
    baseline='middle',
    dx=-5,
    dy=8,
    color=detail,
    fontSize=13,
).encode(
    x=alt.X('date:T'),
    y=alt.Y('value:Q'),
    text='text'
)

text_month = text_value.mark_text(
    align='left',
    baseline='middle',
    dx=-5,
    dy=20,
    color=detail,
    fontSize=13,
).encode(
    text='month'
)


chart_with_text = chart + text_month + text_value

chart_with_text.display()

alt.LayerChart(...)

Save chart

In [88]:
from datetime import date
# Get today's date in formats YYYYMM & YYYYMMDD
yearmonth = date.today().strftime("%Y%m")
today = date.today().strftime("%Y%m%d")

styles.save(chart_with_text, path=f'../../charts/{yearmonth}', name=f'{today}_UK_GDP_Growth_1month', width=380, height=300)

Tweet:

Today’s data released from the ONS reveals UK GDP dropped by 0.3% between September and October. 


#ChartOfTheDay


Make sure you check out our Data Hub to explore, create and share your own charts #dataviz https://buff.ly/3mkZ2je

Extend chart with an overlay line

In [34]:
# Add overlay line chart of 3 month on 3 month growth
line_overlay_colour = '#fff'
chart_line = alt.Chart(df_ed3h[df_ed3h['date'] >= '2022-01-01']).mark_line(color=line_overlay_colour, strokeDash=[4,2], strokeOpacity=0.5).encode(
    x='date:T',
    y='value:Q'
)

chart += chart_line

In [35]:
chart.display()

alt.LayerChart(...)

---

Gross Value Added - Monthly (period on period 1 year ago growth ) :CVM SA

In [23]:
df_ed2r = get_ons_data(series_id='ed2r')

In [27]:
# create chart using Altair
start_date = '2015-01-01'

chart = alt.Chart(df_ed2r[df_ed2r['date'] >= start_date]).mark_line(color=line_colour).encode(
    x=alt.X(
        'date:T',
        axis=alt.Axis(
            labelColor=detail,
            tickColor=detail,
            domainColor=detail,
            domainOpacity=0.5,
            grid=False,
            labelAngle=0,
            tickCount=10,
            tickOpacity=0.5,
            title=None
        )
    ),
    y=alt.Y(
        'value:Q',
        axis=alt.Axis(
            labelColor=detail,
            tickColor=detail,
            domainColor=detail,
            gridColor=detail,
            gridDash=[1, 5],
            gridOpacity=0.5,
            labelPadding=5,
            tickCount=8,
            tickOpacity=0.5,
            ticks=False,
            title='Gross value added, 12-month growth, % | Source: ONS',
            titleAlign='left',
            titleAngle=0,
            titleBaseline='bottom',
            titleColor=detail,
            titleFontSize=12,
            titleOpacity=0.9,
            titleX=0,
            titleY=-7
        ),
        scale=alt.Scale(zero=False, padding=30)
    )
).properties(
    width = 400,
    height = 300,
    title = 'UK GDP Growth'
)

# Add a point on at the last data point
# First filter the data to get the last data point
# Then add a point on the chart
last_point = df_ed2r[df_ed2r['date'] == df_ed2r['date'].max()].copy()

point = alt.Chart(last_point).mark_point(color=line_colour, size=60, opacity=0.9, filled=True).encode(
    x=alt.X('date:T'),
    y=alt.Y('value:Q')
)

# Now let's add text to the point
# add a column to `last_point` dataframe with the text we want to display
last_point['text'] = last_point['value'].apply(lambda x: f'{x:.1f}%')

text_value = point.mark_text(
    align='left',
    baseline='middle',
    dx=7,
    dy=0,
    color=detail,
    fontSize=14,
).encode(
    text='text'
)

text_month = point.mark_text(
    align='left',
    baseline='middle',
    dx=7,
    dy=-16,
    color=detail,
    fontSize=14,
).encode(
    text='month'
)

chart = chart + point + text_month + text_value

# add a line at y=0
chart += alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color=detail, size=1.5, opacity=0.5).encode(y='y')


chart = chart.configure(
    font='Circular Std',
    background=background
).configure_title(
    anchor='start',
    dx=20,
    fontSize=12,
    subtitleFontSize=12,
    color=detail,
    subtitleColor='#000000'
).configure_view(
    stroke=None
)

chart.display()



alt.LayerChart(...)

In [16]:
# Convert chart to dictionary
chart_dict = chart.to_dict()

# Modify width and height directly (Altair will set continuousWidth and height by default)
chart_dict['width'] = 350
chart_dict['height'] = 280

# Convert the dictionary back to JSON
vega_spec = json.dumps(chart_dict, indent=2)

# Write the JSON to a file
with open('../../charts/202310/20231012_UK_GDP_Growth_12month.json', 'w') as f:
    f.write(vega_spec)

---